In [1]:
# ===========================================
# 02_GK: session -> daily, save per-player
# ===========================================
import pandas as pd
import numpy as np
import os

CLEAN_PATH = "/workspace/data/00/StatsAllGroup_cleaned.parquet"
OUT_BASE = "/workspace/data/02_gk"
PLAYER_DIR = f"{OUT_BASE}/players"
os.makedirs(PLAYER_DIR, exist_ok=True)

df_clean = pd.read_parquet(CLEAN_PATH)
df_clean = df_clean[df_clean["position_group"] == "GK"].copy()

print(f"✅ Loaded GK data: {df_clean.shape[0]} rows, {df_clean['athlete_name'].nunique()} athletes")

df = df_clean.copy()
df["date_"] = pd.to_datetime(df["date_"]).dt.normalize()

# Match Day フラグ（FPと同じ）
df["is_match_day"] = df["activity_name"].str.contains(
    r"vs|VS|tm|TM|match|試合", case=False, na=False
).astype(int)

# 数値列の集計定義
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ["start_time", "end_time"]]

agg_dict = {}
for col in numeric_cols:
    if col == "mean_heart_rate":
        agg_dict[col] = "mean"
    elif col in ["max_heart_rate", "max_vel"]:
        agg_dict[col] = "max"
    elif col == "min_heart_rate":
        agg_dict[col] = "min"
    else:
        agg_dict[col] = "sum"

agg_dict["is_match_day"] = "max"

# チーム全体の Match Day（GKデータからでも取れるが、無い場合に備える）
match_days = (
    df.loc[df["is_match_day"] == 1, "date_"]
      .drop_duplicates()
      .sort_values()
      .to_numpy()
      .astype("datetime64[D]")
)

daily_raw = (
    df.groupby(["athlete_id", "date_"])
      .agg(agg_dict)
      .reset_index()
      .sort_values(["athlete_id", "date_"])
)

athlete_master = (
    df[["athlete_id", "athlete_name"]]
    .drop_duplicates(subset=["athlete_id"])
)

daily_raw = daily_raw.merge(athlete_master, on="athlete_id", how="left")

# md_offset
if len(match_days) == 0:
    daily_raw["md_offset"] = np.nan
else:
    dates = daily_raw["date_"].to_numpy().astype("datetime64[D]")
    diff = (dates[:, None] - match_days[None, :]).astype("timedelta64[D]").astype(int)
    daily_raw["md_offset"] = diff[np.arange(len(diff)), np.abs(diff).argmin(axis=1)]

def to_md_phase(x):
    if pd.isna(x): return "NONE"
    if x == 0: return "MD"
    if x == 1: return "MD+1"
    if x == -1: return "MD-1"
    if x == 2: return "MD+2"
    if x == -2: return "MD-2"
    return "OTHER"

daily_raw["md_phase"] = daily_raw["md_offset"].apply(to_md_phase)

# 一意性
daily_raw = daily_raw.drop_duplicates(subset=["athlete_id", "date_"])
assert daily_raw[["athlete_id", "date_"]].duplicated().sum() == 0

# 保存（全体）
os.makedirs(OUT_BASE, exist_ok=True)
daily_raw.to_parquet(f"{OUT_BASE}/gk_daily.parquet", index=False)
daily_raw.to_csv(f"{OUT_BASE}/gk_daily.csv", index=False)

print("💾 Saved GK daily:")
print(" -", f"{OUT_BASE}/gk_daily.parquet")

# FPと同じく “dynamic/static/acwr” に分ける（GKではまず static を重視）
meta_cols = ["athlete_id", "athlete_name", "date_", "md_offset", "md_phase", "is_match_day"]

# GK用 static候補（存在する列だけ使う）
GK_COLS = [
    "total_dive_load_centre", "total_dive_load_left", "total_dive_load_right",
    "dive_right_count", "dive_left_count", "dive_centre_count",
    "total_dives_centre", "total_dives_left", "total_dives_right",
    "total_time_to_feet_centre", "total_time_to_feet_left", "total_time_to_feet_right"
]
gk_cols_present = [c for c in GK_COLS if c in daily_raw.columns]

# まずはGK static = GK_COLS（＋必要ならHR系も後で足せる）
df_gk_static = daily_raw[meta_cols + gk_cols_present].copy()

# per-player 保存（GKはとりあえず static のみでOK）
for athlete_id, sub in df_gk_static.groupby("athlete_id"):
    out_dir = f"{PLAYER_DIR}/{athlete_id}"
    os.makedirs(out_dir, exist_ok=True)
    sub.to_parquet(f"{out_dir}/{athlete_id}_gk_static.parquet", index=False)

print("💾 Saved per-player GK static under:", PLAYER_DIR)
print("GK cols used:", gk_cols_present)


✅ Loaded GK data: 2523 rows, 4 athletes
💾 Saved GK daily:
 - /workspace/data/02_gk/gk_daily.parquet
💾 Saved per-player GK static under: /workspace/data/02_gk/players
GK cols used: ['total_dive_load_centre', 'total_dive_load_left', 'total_dive_load_right', 'dive_right_count', 'dive_left_count', 'dive_centre_count', 'total_dives_centre', 'total_dives_left', 'total_dives_right', 'total_time_to_feet_centre', 'total_time_to_feet_left', 'total_time_to_feet_right']
